In [ ]:
import mbuild as mb
import foyer
import unyt as u

## Prepare recipes with mBuild

### Exercise 1

In [ ]:
class _CH2(mb.Compound):
    def __init__(self):
        super().__init__()
        self.name = "_CH2"
        """Create a partilce name _CH2 and add it to self."""
        # Enter your code here
        
        
        """Create a port attached to the _CH2 particles, label as 'up'."""
        port_up = mb.Port(anchor=_ch2, orientation=[0, 0, 1], separation=0.07)
        self.add(port_up, "up")
        
        
        """Create a port attached to the _CH2 particles, label as 'down',
        with same separation but opposite orientation as 'up port'
        """
        # Enter your code here

        
class _CH3(mb.Compound):
    def __init__(self):
        super().__init__()
        self.name = "_CH3"
        
        """Create a partilce name _CH3 and add it to self."""
        # Enter your code here
        
        
        """Create a port attached to the _CH3 particles, label as 'up'.
        Use the separation value of the _CH2's ports as above.
        """

            

### Exercise 2a

In [ ]:
class CG_Alkane(mb.Compound):
    def __init__(self, n):
        super().__init__()
        self.name = "CG_Alkane"
    
        if n == 1: 
            _ch4 = mb.Compound(name="_CH4")
            self.add(_ch4, "_CH4")
        elif n == 2:
            _ch3a = _CH3() 
            _ch3b = _CH3() 
            self.add(_ch3a, "_CH3_a")
            self.add(_ch3b, "_CH3_b")
            mb.force_overlap(move_this=_ch3a, 
                             from_positions=_ch3a["up"], 
                             to_positions=_ch3b["up"])
        else:
            """ Create a _CH3 particle and add it to self with label 'link0'"""
            # Enter your code here

            
            for i in range(n - 2):
                """ Create a _CH2 particle and add it to self with label f'link{i}'.
                Then force_overlap it (from Port "down") with 
                the previous link (to Port "up").
                """
                
                
            """ Create another _CH3 particle and add it to self with label f'link{i}.
            Then force_overlap it with the previous link.
            """
            # Enter your code here
            


### Exercise 2b

In [4]:
from mbuild.lib.recipes import Polymer

help(Polymer)
"""Create a Polymer() object with _CH2 monomer and ends with 2 _CH3."""
# Enter your code here

"""Build the just-created object with length 5"""
# Enter your code here


Help on class Polymer in module mbuild.lib.recipes.polymer:

class Polymer(mbuild.compound.Compound)
 |  Polymer(monomers=None, end_groups=None)
 |  
 |  Connect one or more components in a specified sequence.
 |  
 |  Attributes
 |  ----------
 |  monomers : list of mbuild.Compounds
 |      The compound(s) to replicate. Add to this list using the add_monomers
 |      method.
 |  end_groups : list of mbuild.Compounds
 |      The compound to cap the end of the polymer. Add to this list using the
 |      add_end_groups method.
 |  
 |  Methods
 |  -------
 |  add_monomer(monomer, indices, separation, port_labels, orientation, replace)
 |      Use to add a monomer compound to Polymer.monomers
 |  
 |  add_end_groups(compound, index, separation, orientation, replace)
 |      Use to add an end group compound to Polymer.end_groups
 |  
 |  build(n, sequence)
 |      Use to create a single polymer compound. This method uses the compounds
 |      created by calling the add_monomer and add_end_

'Build the just-created object with length 5'

### Exercise 3

In [ ]:
"""Create a pentane using the CG_Alkane recipe above."""
# Enter your code here

"""Create a box of pentane using the mb.packing.fill_box() method.
Let's fill the box of size [3.2, 3.2, 3.2] with a padding of 0.2 
with 150 compounds."""
# Enter your code here

"""Perform a brief energy minimization of the pentane box created above
with the trappe forcefield"""
# Enter your code here
help(mb.Compound.energy_minimize)

## Atomtyping and Parameterization with Foyer
### Exercise 4

In [12]:
import foyer 
from foyer import Forcefield

"""Load the TRAPPE-UA forcefield (name "trappe-ua")"""
# Enter your code here

"""Use foyer to apply forcefields to a pentane and a box of pentane"""
# Enter your code here

### Exercise 5a - Simulation with MosDeF Cassandra

In [ ]:
import mosdef_cassandra as mc

# Create box and species list
species_list = [typed_pentane]
box_list = [pentane_box]
mols_in_boxes = [[150]]

system = mc.System(box_list, species_list, mols_in_boxes=mols_in_boxes)
moveset = mc.MoveSet("npt", species_list)

moveset.prob_volume = 0.010

thermo_props = [
    "energy_total",
    "energy_intervdw",
    "pressure",
    "volume",
    "nmols",
    "mass_density",
]

custom_args = {
    "run_name": "equil",
    "charge_style": "none",
    "rcut_min": 2.0 * u.angstrom,
    "vdw_cutoff": 14.0 * u.angstrom,
    "units": "sweeps",
    "steps_per_sweep": 450,
    "coord_freq": 50,
    "prop_freq": 10,
    "properties": thermo_props,
}


mc.run(
    system=system,
    moveset=moveset,
    run_type="equilibration",
    run_length=2400,
    pressure=14.02 * u.bar,
    temperature=372 * u.K,
    **custom_args,
)

### Exercise 5b - Simulation with HOOMD Blue

In [ ]:
"""Save out files needed to perform a GROMACS simluation (.gro and .top)"""

In [ ]:
nvt_mdp = """
integrator          = md
nsteps              = 100000
dt                  = 0.001

comm-mode           = Linear

nstxout             = 10000
nstvout             = 10000
nstenergy           = 1000
nstlog              = 1000

cutoff-scheme       = Verlet
ns_type             = grid
nstlist             = 10
pbc                 = xyz 

vdwtype         = Cut-off
vdw-modifier    = None
rvdw            = 1.4 

coulombtype             = Cut-off
coulomb-modifier        = None
rcoulomb                = 1.4 

tcoupl              = nose-hoover
tc-grps             = System
tau_t               = 1 
ref_t               = 372.0

pcoupl              = no

DispCorr            = EnerPres
"""
with open("gmx_sim/nvt.mdp", "w") as f:
    f.write(nvt_mdp)

In [ ]:
%cd gmx_sim
!gmx grompp -f nvt.mdp -o nvt.tpr -c init.gro -p init.top --maxwarn 1
!gmx mdrun -v -deffnm nvt -s nvt.tpr -cpi nvt.cpt
%cd ..

### Exercise 6

In [ ]:
import numpy as np
import pylab as plt

data = np.genfromtxt("equil.out.prp", skip_header=3)

plt.rcParams['font.family'] = "DIN Alternate"
font = {'family' : 'DIN Alternate',
        'weight' : 'normal',
        'size'   : 12}

fig, ax = plt.subplots(1, 1)

ax.spines["bottom"].set_linewidth(3)
ax.spines["left"].set_linewidth(3)
ax.spines["right"].set_linewidth(3)
ax.spines["top"].set_linewidth(3)

ax.title.set_text('Control plot')
ax.set_xlabel(r'MC sweeps')
ax.set_ylabel('Density $(kg / m{^3})$')
ax.yaxis.tick_left()
ax.yaxis.set_label_position('left')
ax.axhline(y=541, color='r', linestyle='-', label='~TraPPE-UA Density')

ax.plot(data[:, 0], data[:,-1], "-", color='lightgray', label='Density')
ax.legend(loc="best")
plt.show()